In [2]:
# Importing libraries
import os
import requests
import time
from time import sleep
from random import uniform
from datetime import date, timezone, datetime, timedelta
import csv

import sys
import re
import numpy as np
import pandas as pd
import json

from dotenv import load_dotenv
load_dotenv()

from pymongo import MongoClient
from pymongo.collection import Collection

from gspread_formatting import *
import gspread
from gspread_pandas import Spread

In [2]:
DB_NAME     = os.environ.get("DB_NAME")
COLL_NAME   = os.environ.get("COLL_NAME")
MONGO_URL   = os.environ.get("M")

In [5]:
# Get data from factchecking sites MongoDB
def initialize_mongo(MONGOURL):
       
    cli = MongoClient(MONGOURL)
    db = cli[DB_NAME]
    coll = db[COLL_NAME]
    if coll.count_documents({}) >= 0:
        return coll 
    else:
        print("Error accessing Mongo collection")

In [6]:
coll = initialize_mongo(MONGOURL)

In [7]:
coll.count_documents({})

15979

### Get Count of Stories Added in the Last Week

In [13]:
def get_weekly_data(coll):
    
    end_date = date.today()
    start_delta = timedelta(days=7)
    start_date = end_date-start_delta
    
    pipeline = [
        {"$project":{"date_accessed":"$date_accessed", "date_updated":"$date_updated", 'postID': "$postID",'postURL': "$postURL",
                     "headline": "$headline", "docs": "$docs", "author": "$author", "domain": "$domain",
                     "date": {"$dateFromString": {"dateString": "$date_updated"}}}},
        {"$match": {"date": {"$gte":datetime(start_date.year, start_date.month, start_date.day, 0, 0), 
                             "$lt": datetime(end_date.year, end_date.month, end_date.day, 0, 0)}}},
        {"$count" : "count"}
        
    ]
    
    docs = coll.aggregate(pipeline)
    return docs,start_date,end_date

In [30]:
def get_weekly_count_v2(coll):
    
    c = 0
    result = []
    docs,start_date,end_date = get_weekly_data(coll)
    
    weekly_dict = {'start_date' : str(start_date),'end_date':str(end_date),'count' : list(docs)[0]['count']} 
    
    return weekly_dict

### Get Count of Stories per fact checking site

In [16]:
def get_stories_by_domain(coll):
    pipeline = [
        {"$project":{"date_accessed":"$date_accessed", "date_updated":"$date_updated", 'postID': "$postID",'postURL': "$postURL",
                     "headline": "$headline", "docs": "$docs", "author": "$author", "domain": "$domain",
                     "date": {"$dateFromString": {"dateString": "$date_updated"}}}},
    ]
    
    docs = coll.aggregate([{"$group" : {"_id": "$domain", "count":{"$sum":1}}}])
    return docs

-- run_date,domain,count

In [26]:
def get_domain_count(coll):
    
    docs = get_stories_by_domain(coll)
    
    data = pd.DataFrame(docs)
    
    data.columns = ['domain','count']
    data['run_date'] = str(date.today())
    
    data.insert(0, 'run_date', data.pop('run_date'))

    return data
   

In [27]:
get_domain_count(coll)

,run_date,domain,count
0,2022-04-08,en.youturn.in,172
1,2022-04-08,newschecker.in/pa,58
2,2022-04-08,newschecker.in/ml,58
3,2022-04-08,factcrescendo.com,223
4,2022-04-08,newschecker.in/hi,94
5,2022-04-08,thelogicalindian.com,103
6,2022-04-08,https://newschecker.in/bn,17
7,2022-04-08,newschecker.in/bn,60
8,2022-04-08,https://newschecker.in/gu,8
9,2022-04-08,newschecker.in/mr,58


### Write to Google doc

In [28]:
def upload_to_spreadsheet(coll,worksheet_name,EMAIL='techtattle6@gmail.com',):
    
    service = gspread.service_account('cli_secret.json')
    workbook_name = 'FCDBStatus_Public'
    sh = service.open(workbook_name)
    
    sh.share(EMAIL, perm_type='user', role='writer')
        
    if worksheet_name == "updated_count":
        
        try:
            
            sheet = sh.worksheet(worksheet_name)
    
        except Exception as exc:

            print(f'Added Worksheet ....')
            sheet = sh.add_worksheet(title=worksheet_name, rows=1, cols=20)

        data = get_weekly_count_v2(coll)

        print(f"Updating Weekly count sheet ....")

        max_rows = len(sheet.get_all_values())


        row_value = str(max_rows+1)
        start_date_pos = 'A' + row_value
        end_date_pos = 'B'+ row_value
        count_pos = 'C' + row_value

        sheet.update(start_date_pos,data['start_date'])
        sheet.update(end_date_pos,data['end_date'])
        sheet.update(count_pos,data['count'])
        
        print(f"Finish updating Weekly count sheet ....")
    
    elif worksheet_name == "domain_count":
        
        try:
            
            sheet = sh.worksheet(worksheet_name)
    
        except Exception as exc:

            print(f'Added New Worksheet ....')
            sheet = sh.add_worksheet(title=worksheet_name, rows=1, cols=20)
            
        
        data = get_domain_count(coll)
        
        print(f"Updating Domain count sheet ....")
        
        max_rows = len(sheet.get_all_values())
        
        start_value = 'A' + str(max_rows + 1)
        end_value =   'C' + str(max_rows + data.shape[0])
        
        update_pos = start_value + ':' + end_value
        
        sheet.update(update_pos,data.values.tolist())

        print(f"Finish updating Weekly count sheet ....")
        
    else:
        
        print(f"No {worksheet_name} sheet found ...")


In [31]:
upload_to_spreadsheet(coll,'updated_count')

Updating Weekly count sheet ....
Finish updating Weekly count sheet ....


In [32]:
upload_to_spreadsheet(coll,'domain_count')

Updating Domain count sheet ....
Finish updating Weekly count sheet ....
